In [280]:
import numpy as np

l = 9

board = np.array([[0,0,0,0,2,0,0,0,4],
                       [0,0,2,1,7,0,0,0,0],
                       [0,0,0,6,0,5,7,0,0],
                       [0,0,0,3,0,0,6,0,8],
                       [0,9,0,0,0,0,0,1,0],
                       [0,6,7,2,0,9,0,0,0],
                       [0,0,9,0,0,0,0,0,3],
                       [1,0,8,9,0,0,4,0,0],
                       [0,0,0,0,6,0,0,0,0]])

file = np.genfromtxt("SudokuEasy50.txt",int,delimiter=1)
boards = []
for line in file:
    boards.append(np.reshape(line,(9,9)))

def unique(l):
    ll = np.array(list(set(l)))
    return np.delete(ll,np.where(ll==0))

def solve(board,depth):
    board = board.copy()
    
    
    possibles = np.zeros([9,9,9])
    for y in range(l):
        for x in range(l):
            if board[x,y]==0:
                possibles[x,y] = np.arange(9)+1
    
    K = np.sum(board==0)
    count = 0
    while K > 0:
        if count > depth:
            return False,board
        for y in range(l):
            for x in range(l):
                
                # Only fill in if not already
                if board[x,y] != 0:
                    continue
                    
                #find row and column
                r,c = board[:,y],board[x,:]
                
                #find local square
                sx,sy = int(np.floor(x/3)*3),int(np.floor(y/3)*3)
                tile = board[sx:sx+3,sy:sy+3].ravel()
                
                #combine all
                comb = unique(np.hstack((r,c,tile)).ravel())
                
                #remove possibilites if already found
                for c in comb:
                    possibles[x,y][np.where(possibles[x,y]==c)] = 0
                
                #Find other possibilities in the row and coloumn
                r,c = possibles[np.arange(l)!=x,y].ravel(),possibles[x,np.arange(l)!=y].ravel()
                r,c = unique(r),unique(c)
                
                #And the possibilities in the square
                square = np.empty((8,9))
                loopings = 0 #slam
                for xx in range(sx,sx+3):
                    for yy in range(sy,sy+3):
                        if (xx == x) and (yy == y):
                            continue
                        square[loopings] = possibles[xx,yy] 
                        loopings+=1
                square = unique(square.ravel())
            
                #If you are the only one in row/coloum/square that can have number, you take it
                us = unique(possibles[x,y])
                for u in us:
                    if (u not in r) or (u not in c) or (u not in square):
                        tf = np.where(possibles[x,y]==u)
                        mask = np.ones(possibles[x,y].shape, dtype='bool')
                        mask[tf] = False
                        possibles[x,y][mask] = 0
                
                #find the possible numbers for this position and fill in if there is only one left
                us = unique(possibles[x,y])
                if(len(us)==1):
                    board[x,y] = us[0]
                    possibles[x,y] = np.zeros(9)
                elif(len(us)==0):
                    print("Something wrong at ",x,y)
                    
        #Check if done
        K = np.sum(board==0)
        count += 1

    return True,board

In [281]:
print(board)
_, board = solve(board,100)
board

[[0 0 0 0 2 0 0 0 4]
 [0 0 2 1 7 0 0 0 0]
 [0 0 0 6 0 5 7 0 0]
 [0 0 0 3 0 0 6 0 8]
 [0 9 0 0 0 0 0 1 0]
 [0 6 7 2 0 9 0 0 0]
 [0 0 9 0 0 0 0 0 3]
 [1 0 8 9 0 0 4 0 0]
 [0 0 0 0 6 0 0 0 0]]


array([[9, 7, 6, 8, 2, 3, 1, 5, 4],
       [5, 3, 2, 1, 7, 4, 8, 6, 9],
       [4, 8, 1, 6, 9, 5, 7, 3, 2],
       [2, 1, 4, 3, 5, 7, 6, 9, 8],
       [3, 9, 5, 4, 8, 6, 2, 1, 7],
       [8, 6, 7, 2, 1, 9, 3, 4, 5],
       [6, 2, 9, 7, 4, 1, 5, 8, 3],
       [1, 5, 8, 9, 3, 2, 4, 7, 6],
       [7, 4, 3, 5, 6, 8, 9, 2, 1]])